## Memory for Recurrent Neural Networks




In [1]:
import pandas as pd
import numpy as np

# onehotencoding
def encode(sequence, n_unique):
	encoded = list()
	for value in sequence:
		row = [0.0 for x in range(n_unique)]
		row[value] = 1.0
		encoded.append(row)
	return encoded
 
seq1 = [0, 1, 2, 3, 4, 5]
encoded = encode(seq1, 6)
for v in encoded:
	print(v)

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [2]:
# create pairs of encoded vectors, returns X, y

def train(encoded):
	X,y = list(),list()
	for i in range(2, len(encoded)):
		X.append(encoded[i-2])  # input 
		y.append(encoded[i])    # output
	return X, y

In [3]:
seq = [3, 2, 1, 0, 4, 5, 17, 10, 20, 30, 23, 24, 6, 26, 30, 12, 14, 30, 22, 29]
encoded = encode(seq, np.array(seq).max()+1)
X, y = train(encoded)
for i in range(len(X)):
	print(X[i], y[i])

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [4]:
# preprocess sequences to use in LSTM

def preprocessing(sequence, n_unique):
	# onehotencoding
	encoded = encode(sequence, n_unique)
	# construct train set
	X,y = train(encoded)
	# convert and reshape for use in LSTM
	X, y = pd.DataFrame(X), pd.DataFrame(y)
	X = X.values
	X_new = X.reshape(X.shape[0], 1, X.shape[1])
	Y_new = y.values
	return X_new, Y_new

In [5]:
sequence_X, sequence_Y = preprocessing(seq, np.array(seq).max()+1)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# define LSTM configuration
n_neurons = 64
n_batch = 1
n_epoch = 250
n_features = np.array(seq).max()+1

# construct a model
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(1, 1, 31), stateful=True))
model.add(Dense(n_features, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [7]:
# train LSTM
for i in range(n_epoch):
	model.fit(sequence_X, sequence_Y, epochs=1, batch_size=1, verbose=2, shuffle=False)

18/18 - 1s - loss: 0.6871
18/18 - 0s - loss: 0.6503
18/18 - 0s - loss: 0.4203
18/18 - 0s - loss: 0.1659
18/18 - 0s - loss: 0.1343
18/18 - 0s - loss: 0.1278
18/18 - 0s - loss: 0.1250
18/18 - 0s - loss: 0.1235
18/18 - 0s - loss: 0.1224
18/18 - 0s - loss: 0.1216
18/18 - 0s - loss: 0.1210
18/18 - 0s - loss: 0.1205
18/18 - 0s - loss: 0.1201
18/18 - 0s - loss: 0.1197
18/18 - 0s - loss: 0.1193
18/18 - 0s - loss: 0.1189
18/18 - 0s - loss: 0.1186
18/18 - 0s - loss: 0.1183
18/18 - 0s - loss: 0.1179
18/18 - 0s - loss: 0.1176
18/18 - 0s - loss: 0.1173
18/18 - 0s - loss: 0.1169
18/18 - 0s - loss: 0.1166
18/18 - 0s - loss: 0.1162
18/18 - 0s - loss: 0.1158
18/18 - 0s - loss: 0.1154
18/18 - 0s - loss: 0.1150
18/18 - 0s - loss: 0.1146
18/18 - 0s - loss: 0.1141
18/18 - 0s - loss: 0.1136
18/18 - 0s - loss: 0.1131
18/18 - 0s - loss: 0.1126
18/18 - 0s - loss: 0.1121
18/18 - 0s - loss: 0.1116
18/18 - 0s - loss: 0.1110
18/18 - 0s - loss: 0.1104
18/18 - 0s - loss: 0.1098
18/18 - 0s - loss: 0.1092
18/18 - 0s -

In [8]:
result = np.argmax(model.predict(sequence_X, batch_size=n_batch), axis=-1) 

# test LSTM on sequence 2
print('Sequence')
print(seq)
for i in range(len(result)):
	print('X = %.1f, prediction = %.1f' % (seq[i], result[i]))

Sequence
[3, 2, 1, 0, 4, 5, 17, 10, 20, 30, 23, 24, 6, 26, 30, 12, 14, 30, 22, 29]
X = 3.0, prediction = 1.0
X = 2.0, prediction = 0.0
X = 1.0, prediction = 4.0
X = 0.0, prediction = 5.0
X = 4.0, prediction = 17.0
X = 5.0, prediction = 10.0
X = 17.0, prediction = 20.0
X = 10.0, prediction = 30.0
X = 20.0, prediction = 23.0
X = 30.0, prediction = 24.0
X = 23.0, prediction = 6.0
X = 24.0, prediction = 26.0
X = 6.0, prediction = 30.0
X = 26.0, prediction = 12.0
X = 30.0, prediction = 14.0
X = 12.0, prediction = 30.0
X = 14.0, prediction = 22.0
X = 30.0, prediction = 29.0
